In [1]:

from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


DB_PATH = r"C:\python\dataOnline\anaslo_02\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    }

# 検索キーワードよりホール名取得
SEARCH_WORD = "EXA FIRST"
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    print(f"🔍 '{SEARCH_WORD}' を含むホール名の検索結果:")
    for hall_id, hall_name in results:
        print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals):
    bb_medals = 239.25
    rb_medals = 95.25
    cherry_rate_high = 0.04228
    cherry_rate_low = 0.05847
    replay_rate = 0.411
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"]).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(df.shape)
df.head()

🔍 'EXA FIRST' を含むホール名の検索結果:
 - hall_name: EXA FIRST, hall_id: 108
(57152, 15)


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,grape_rate,Total_rate,month,day,weekday
0,EXA FIRST,2025-04-14,ゴーゴージャグラー3,1001,2617,6,8,-1229,436.2,327.1,6.47,186.9,2025-04,14,0
1,EXA FIRST,2025-04-14,ゴーゴージャグラー3,1002,2473,7,2,-974,353.3,1236.5,5.72,274.8,2025-04,14,0
2,EXA FIRST,2025-04-14,ゴーゴージャグラー3,1003,3898,11,19,-432,354.4,205.2,6.17,129.9,2025-04,14,0
3,EXA FIRST,2025-04-14,ゴーゴージャグラー3,1004,1517,2,4,-968,758.5,379.2,5.96,252.8,2025-04,14,0
4,EXA FIRST,2025-04-14,ゴーゴージャグラー3,1005,9033,39,31,1650,231.6,291.4,5.84,129.0,2025-04,14,0


In [27]:
condition = df["date"].dt.date >= datetime.date.today() - relativedelta(days=1)
df["date"][condition].unique()
print(df["model_name"].unique())
print(df["model_name"][condition].unique())
print(df["unit_no"].nunique())
print(df["unit_no"][condition].nunique())

['マイジャグラーV' 'ファンキージャグラー2' 'ミスタージャグラー' 'ゴーゴージャグラー3' 'ウルトラミラクルジャグラー'
 'アイムジャグラーEX-TP' 'ジャグラーガールズ' 'ハッピージャグラーVIII' 'SミスタージャグラーKK']
['マイジャグラーV' 'ファンキージャグラー2' 'ミスタージャグラー' 'ゴーゴージャグラー3' 'ウルトラミラクルジャグラー'
 'アイムジャグラーEX-TP' 'ジャグラーガールズ']
152
152


## MONTH_ANALYSIS

In [10]:
# 日ごとにモデル別で表示
SHEET_NAME = "HOLE_ANALYSIS"
MODEL_NAME = "マイジャグラーV"
df_tmp = df[(df["model_name"] == MODEL_NAME)].copy()
# メダル
summary = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="month",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
summary.head()

month               2024-04  2024-05  2024-06  2024-07  2024-08  2024-09  \
model_name unit_no                                                         
マイジャグラーV   1001      2895.0   4620.0   1012.0  -7425.0  -3903.0   3233.0   
           1002     13950.0   5351.0   8095.0   7651.0   2323.0   8350.0   
           1003     14057.0  12475.0   9104.0   4498.0   1771.0   6580.0   
           1004      4949.0   8723.0   3594.0   6261.0   4198.0   -907.0   
           1005     11637.0   4167.0  11100.0   5600.0    820.0     51.0   

month               2024-10  2024-11  2024-12  2025-01  2025-02  2025-03  \
model_name unit_no                                                         
マイジャグラーV   1001      2371.0  -7162.0  -9032.0   1606.0 -10442.0  -7233.0   
           1002      5227.0  -7894.0    591.0  13541.0  -1870.0   5959.0   
           1003      -877.0   8672.0   6747.0  -4302.0   4553.0   5974.0   
           1004      7221.0   1500.0  -2992.0   8438.0   4868.0   5599.0   
           1005     17447.0  -2571.0  -3268.0   5252.0    362.0   1950.0   

month               2025-04  Total  
model_name unit_no                  
マイジャグラーV   1001     -1672.0 -31132  
           1002      3149.0  64423  
           1003      4924.0  74176  
           1004     -3734.0  47718  
           1005      6048.0  58595

## HOLE_ANALYSIS

In [7]:
# 日ごとにモデル別で表示
SHEET_NAME = "HOLE_ANALYSIS"
df_tmp = df.copy()
# メダル
medals = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# RB＿RATE
game = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna("")
game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
# WRITE
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, game, row=next_row, include_index=True)

## UNIT_ANALYSIS

In [ ]:
# マシン別メダル
SHEET_NAME = "UNIT_ANALYSIS"
MACHINE_NAME = "マイジャグラーV"
# df_tmp = df[df["model_name"] == MACHINE_NAME].copy()
df_tmp = df.copy()
# メダル数
medals = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="medals",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# medals["rank"] = medals["Total"].rank(method="min", ascending=False).astype(int)
medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
medals.shape
# RB_rate 作成
game = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="RB",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
rb_rate = (game / rb).round(1)
# rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
rb_rate = rb_rate.fillna(0)
rb_rate.shape

# 回転数
games = df_tmp.pivot_table(
    index=["model_name", "unit_no"],
    columns="day",
    values="game",
    aggfunc="sum",
    margins=True,
    margins_name="Total",
)
# games["rank"] = games["Total"].rank(method="min", ascending=False).astype(int)
games.columns = pd.MultiIndex.from_product([["GAMES"], games.columns])
games.head()

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()
set_with_dataframe(sheet, medals, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 4  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)
next_row = len(existing) + 6  # +2: ヘッダー + 次の空行
set_with_dataframe(sheet, games, row=next_row, include_index=True)
existing = get_as_dataframe(sheet, evaluate_formulas=True)

## DAY_DETAIL_N

In [5]:
# 日付指定
DAY = 15
SHEET_NAME = f"DAY_DETAIL_{DAY}"

next_row = 1
spreadsheet = client.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)

models = [
    "マイジャグラーV",
    # "ゴーゴージャグラー3",
    # "アイムジャグラーEX-TP",
    # "ファンキージャグラー2",
]

for model in models:

    df_tmp = df[(df["day"] == DAY) & (df["model_name"] == model)].copy()
    df_tmp["date"] = df_tmp["date"].dt.strftime("%Y-%m-%d")

    # メダル数
    desired_order = ["medals", "game", "Total_rate", "RB_rate", "grape_rate"]
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values=desired_order,
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    # カラム順序を入れ替える（swaplevel）＆ ソート
    medals = medals.replace([np.inf, -np.inf], np.nan)
    medals = medals.fillna(0)
    medals.columns = medals.columns.swaplevel(0, 1)
    medals = medals.sort_index(axis=1, level=0, ascending=False)  # 日付ごとに並び替え

    # medals.drop(labels="Total", level=0, inplace=True)
    # medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    display(medals.head())

    # RB_RATE
    # game = df_tmp.pivot_table(
    #     index=["model_name", "unit_no"],
    #     columns="date",
    #     values="game",
    #     aggfunc="sum",
    #     margins=True,
    #     margins_name="Total",
    # )
    # rb = df_tmp.pivot_table(
    #     index=["model_name", "unit_no"],
    #     columns="date",
    #     values="RB",
    #     aggfunc="sum",
    #     margins=True,
    #     margins_name="Total",
    # )
    
    # rb_rate = (game / rb).round(1)
    # medals.drop(labels="Total", level=0, inplace=True)
    # rb_rate["rank"] = rb_rate["Total"].rank(method="min", ascending=True).astype(int)
    # rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    # rb_rate = rb_rate.replace([np.inf, -np.inf], np.nan)
    # rb_rate = rb_rate.fillna(0)
    # print(f"{model}_{rb_rate.shape}")

    # スプレッドシート追記
    sheet.clear()
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5  # 追記行
    # # set_with_dataframe(sheet, rb_rate, row=next_row, include_index=True)
    # existing = get_as_dataframe(sheet, evaluate_formulas=True)
    # next_row += medals.shape[0] + 5  # 追記行

WorksheetNotFound: DAY_DETAIL_15

In [ ]:
medals.columns

## DAY n

In [6]:
def medals_and_rb_rate(machine_name, year, month, day, period=3):
    base_date = datetime.date(year, month, day)
    start_date = base_date + datetime.timedelta(days=-1)
    end_date = base_date + datetime.timedelta(days=-period)
    df_tmp = df[
        (df["model_name"] == machine_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ]

    # メダル
    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    medals.tail()

    # RB_RATE
    df_tmp = df[(df["model_name"] == machine_name) & (df["date"].dt.date == base_date)]
    game = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="game",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    bb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="BB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="RB",
        aggfunc="sum",
        # margins=True,
        # margins_name="Total",
    )
    rb_rate = (game / rb).round(1)
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    rb_rate.head()
    
    total_rate = (game / (bb+rb)).round(1)
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])

    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
    bb.columns = pd.MultiIndex.from_product([["BB"], bb.columns])
    rb.columns = pd.MultiIndex.from_product([["RB"], rb.columns])

    # データを表示（長い場合は全表示）
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_columns", None)
    medal_rb_rate = pd.concat([medals, game, total_rate, rb_rate, bb, rb], axis=1)
    medal_rb_rate = medal_rb_rate.replace([np.inf, -np.inf], np.nan)
    medal_rb_rate = medal_rb_rate.fillna(0)

    medal_rb_rate = medal_rb_rate[medal_rb_rate["MEDALS"]["rank"] <= 10]

    return medal_rb_rate


# N_DAYS_AGO
MACHINE_NAME = "マイジャグラーV"

medal_rb_rate = medals_and_rb_rate(MACHINE_NAME, 2025, 4, 14)
medal_rb_rate.head(10)

MEDALS                      \
date               2025-04-11 00:00:00 2025-04-12 00:00:00   
model_name unit_no                                           
マイジャグラーV   1001                   -453               -1112   
           1029                  -1371                -924   
           1042                  -1844               -1579   
           1049                   -782                -782   
           1050                  -1006                -935   
           1069                  -2515                -356   
           1082                  -1512                -276   
           1083                  -2568                1088   
           1089                  -1135               -1582   
           1091                  -1135                -915   

                                                                 GAME  \
date               2025-04-13 00:00:00 Total rank 2025-04-14 00:00:00   
model_name unit_no                                                      
マイジャグラーV   1001                   -562 -2127   10                 0.0   
           1029                  -1547 -3842    1              1781.0   
           1042                   -291 -3714    2              7089.0   
           1049                   -571 -2135    9              2363.0   
           1050                   -974 -2915    5              8567.0   
           1069                   -315 -3186    4              6827.0   
           1082                   -488 -2276    8              8576.0   
           1083                   -865 -2345    7              8189.0   
           1089                   -726 -3443    3              6668.0   
           1091                   -562 -2612    6              4896.0   

                            TOTAL_RATE             RB_RATE  \
date               2025-04-14 00:00:00 2025-04-14 00:00:00   
model_name unit_no                                           
マイジャグラーV   1001                    0.0                 0.0   
           1029                  445.2                 0.0   
           1042                  133.8               244.4   
           1049                  181.8               787.7   
           1050                  153.0               342.7   
           1069                  170.7               525.2   
           1082                  138.3               295.7   
           1083                  117.0               264.2   
           1089                  155.1               350.9   
           1091                  148.4               349.7   

                                    BB                  RB  
date               2025-04-14 00:00:00 2025-04-14 00:00:00  
model_name unit_no                                          
マイジャグラーV   1001                    0.0                 0.0  
           1029                    4.0                 0.0  
           1042                   24.0                29.0  
           1049                   10.0                 3.0  
           1050                   31.0                25.0  
           1069                   27.0                13.0  
           1082                   33.0                29.0  
           1083                   39.0                31.0  
           1089                   24.0                19.0  
           1091                   19.0                14.0

In [ ]:
models = ["マイジャグラーV", "ゴーゴージャグラー3", "アイムジャグラーEX-TP", "ファンキージャグラー2"]
today = datetime.date.today()
spreadsheet = client.open_by_key(SPREADSHEET_ID)
# スプレッドシート追記
target_day = 1
for target_day in range(today.day - 1, today.day+1):
    sheet_name = f"DAY{target_day}"
    sheet = spreadsheet.worksheet(sheet_name)
    sheet.clear()
    base = datetime.date(today.year, today.month, target_day)
    dates = [base - relativedelta(months=i) for i in range(7)]
    
    next_row = 1
    for model in models:
        for d in dates:
            medal_rb_rate = medals_and_rb_rate(model, d.year, d.month, d.day)
            set_with_dataframe(sheet, medal_rb_rate, row=next_row, include_index=True)
            existing = get_as_dataframe(sheet, evaluate_formulas=True)
            next_row += medal_rb_rate.shape[0] + 5

    print(f"DAY{target_day} 書き込み完了しました")

## MEDALS_nDAYS_AGO

In [18]:
def get_medals_summary(start_date, end_date, model_name):
    # RB_RATE
    df_tmp = df[
        (df["model_name"] == model_name)
        & (df["date"].dt.date <= start_date)
        & (df["date"].dt.date >= end_date)
    ].copy()

    medals = df_tmp.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values="medals",
        aggfunc="sum",
        margins=True,
        margins_name="Total",
    )
    medals.drop(labels="Total", level=0, inplace=True)
    medals["Rank"] = medals["Total"].rank(method="min", ascending=True).astype(int)
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])

    return medals


today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)
ndays_ago = get_medals_summary(start_date, end_date, model)
ndays_ago.head()

MEDALS                      \
date               2025-04-11 00:00:00 2025-04-12 00:00:00   
model_name unit_no                                           
スマスロ北斗の拳   2207                     76                 -56   
           2208                   -253                  91   
           2209                   -503                 238   
           2210                  -2197               -1488   
           2211                  -1276               -2035   

                                                   
date               2025-04-13 00:00:00 Total Rank  
model_name unit_no                                 
スマスロ北斗の拳   2207                  -1050 -1030   14  
           2208                   -935 -1097   13  
           2209                   -909 -1174   11  
           2210                   1809 -1876    8  
           2211                   -379 -3690    2

In [ ]:
SHEET_NAME = "MEDALS_nDAYS_AGO"

today = datetime.date.today()
start_date = today + datetime.timedelta(days=-1)
end_date = today + datetime.timedelta(days=-3)

models = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
]

sheet = spreadsheet.worksheet(SHEET_NAME)
sheet.clear()

next_row = 1
for model in models:
    medals = get_medals_summary(start_date, end_date, model)
    set_with_dataframe(sheet, medals, row=next_row, include_index=True)
    existing = get_as_dataframe(sheet, evaluate_formulas=True)
    next_row += medals.shape[0] + 5

### 
- 3月5日の過去1日間の出玉ワースト10 → 336以下は5台
- 3月5日の過去2日間の出玉ワースト10 → 336以下は3台
- 3月5日の過去3日間の出玉ワースト10 → 336以下は7台
- 3月5日の過去4日間の出玉ワースト10 → 336以下は6台
### 
- 1月12日の過去1日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去2日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 1月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 1月12日の過去5日間の出玉ワースト10 → 336以下は3台
### 
- 2月12日の過去1日間の出玉ワースト10 → 336以下は3台
- 2月12日の過去2日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去3日間の出玉ワースト10 → 336以下は5台
- 2月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 2月12日の過去5日間の出玉ワースト10 → 336以下は5台
###
- 3月12日の過去1日間の出玉ワースト10 → 336以下は0台
- 3月12日の過去2日間の出玉ワースト10 → 336以下は1台
- 3月12日の過去3日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去4日間の出玉ワースト10 → 336以下は4台
- 3月12日の過去5日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去6日間の出玉ワースト10 → 336以下は3台
- 3月12日の過去7日間の出玉ワースト10 → 336以下は4台

In [47]:
df_tmp = df.copy()
df_tmp[df_tmp["date"].dt.date <= datetime.date(2024, 4, 26)]

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Total_rate,day,month,weekday


In [127]:
print(today - datetime.timedelta(days=347))

2024-05-02
